In [1]:
import tensorflow
from tensorflow import keras
import numpy as np
import pandas as pd
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
import os
import time

Using TensorFlow backend.


In [5]:
name_of_actions = ["clockwise-rotation", "counterclockwise-rotation", "eight", 
                   "flip", "horizontal-rotation", "horizontal-x-shake", "horizontal-y-shake", 
                   "infinity", "vertical-x-shake", "vertical-y-shake"]
exten = ".csv"
type_of_data = ["acc_", "gyro_"]
folder = "split_data/"

step = 45
start_ind = 1
end_ind = 129

model_dir = "saved_model_dir/"
checkpoint_dir = "model_checkpoint/"

In [6]:
def load_answer_file(file):
    dataframe = read_csv(file, header=None, delimiter=',')
    return dataframe.to_numpy()

In [7]:
def load_file(filepath):
    name = filepath[0]
    ind = str(filepath[1])
    dataframe = read_csv(folder + name + '/acc/' + ind + exten, header=None, delimiter=',')
    dataframe_gyro = read_csv(folder + name + '/gyro/' + ind + exten, header=None, delimiter=',')
    dataframe = pd.concat([dataframe, dataframe_gyro], axis=1, sort=False)
    return dataframe.to_numpy() 

In [8]:
def load_group(list_actions):
    datas = list()
    for name in list_actions:
        datas.append(load_file(name))
    return dstack(datas) #3d num array, размерности [samples, time steps, features] -> [ts, feat, sampl]

In [9]:
#checker
list_file_names = [[name_of_actions[0], 1]]
print(load_group(list_file_names).shape)
#first(0) and last(130) files from folder contain not enought lines

(91, 6, 1)


In [10]:
def load_dataset_group(action):
    y = np.empty((0,10), int)
    for place in action:
        name = folder + place[0] + '/answer' + '.csv'
        yy = load_answer_file(name)
        y = np.append(y, yy, axis=0)
    X = load_group(action)
    #y = load_file(folder + action + '/answer' + '.csv') #my bad
    return X, y

In [11]:
def load_dataset(train, test): #get train and test
    trainX, trainy = load_dataset_group(train)
    testX, testy = load_dataset_group(test)
    return trainX, trainy, testX, testy

In [9]:
#checker
#trainX, trainy, testX, testy = load_dataset()

In [10]:
print(test.shape)
print(train.shape)

NameError: name 'test' is not defined

In [12]:
def saved_model_to_lite(model_file, model):
    converter = tf.lite.TFLiteConverter.from_saved_model(model_dir)
    tflite_model = converter.convert()
    open(model_file+".tflite", "wb+").write(tflite_model)

In [13]:
def evaluate_model(trainX, trainy, testX, testy, ind):
    epochs = 15
    batch = 64
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax')) #^layers
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) #accuracy
    
    
    
    if not os.path.isdir('./' + checkpoint_dir + str(ind)):
        try:
            os.mkdir(checkpoint_dir + str(ind))
        except OSError:
            print ("Creation of the directory %s failed" % checkpoint_dir + str(ind))
        else:
            print ("Successfully created the directory %s " % checkpoint_dir + str(ind))
    
    filename = checkpoint_dir + str(ind) + '/' + "cp-{epoch:04d}.ckpt"
    filepath = os.path.dirname(filename)
    
    cp_callback = ModelCheckpoint(filepath=filename,
                                 save_weights_only=True,
                                 verbose=1,
                                 period=5)
    
    
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch, verbose=0, callbacks=[cp_callback])
    
    saved_model_path = checkpoint_dir + str(ind) + '/model.h5'
    model.save(saved_model_path)
    
    a,accuracy = model.evaluate(testX, testy, batch_size=batch, verbose=0)
    return accuracy

In [14]:
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [15]:
def run_experiment(trainX, trainy, testX, testy, repeats=10):
    
    scores = list()
    for r in range(repeats):
        score = evaluate_model(trainX, trainy, testX, testy, r) * 100.0
        print('>#%d: %.3f' % (r+1, score))
        scores.append(score)
    summarize_results(scores)

In [16]:
list_of_all_actions = list()

for name in name_of_actions:
    for ind in range(start_ind, end_ind + 1):
        list_of_all_actions.append([name, ind])

train, test = train_test_split(list_of_all_actions, test_size=0.33, random_state=42)
trainX, trainy, testX, testy = load_dataset(train, test)

In [17]:
trainXm = np.moveaxis(trainX, -1, 0)
testXm = np.moveaxis(testX, -1, 0)

In [38]:
run_experiment(trainXm, trainy, testXm, testy, 10)


Epoch 00005: saving model to model_checkpoint/0/cp-0005.ckpt

Epoch 00010: saving model to model_checkpoint/0/cp-0010.ckpt

Epoch 00015: saving model to model_checkpoint/0/cp-0015.ckpt
>#1: 86.385
Successfully created the directory model_checkpoint/ 1

Epoch 00005: saving model to model_checkpoint/1/cp-0005.ckpt

Epoch 00010: saving model to model_checkpoint/1/cp-0010.ckpt

Epoch 00015: saving model to model_checkpoint/1/cp-0015.ckpt
>#2: 93.192
Successfully created the directory model_checkpoint/ 2

Epoch 00005: saving model to model_checkpoint/2/cp-0005.ckpt

Epoch 00010: saving model to model_checkpoint/2/cp-0010.ckpt

Epoch 00015: saving model to model_checkpoint/2/cp-0015.ckpt
>#3: 96.009
Successfully created the directory model_checkpoint/ 3

Epoch 00005: saving model to model_checkpoint/3/cp-0005.ckpt

Epoch 00010: saving model to model_checkpoint/3/cp-0010.ckpt

Epoch 00015: saving model to model_checkpoint/3/cp-0015.ckpt
>#4: 97.418
Successfully created the directory model_ch

In [43]:
epochs = 15
batch = 64
n_timesteps, n_features, n_outputs = trainXm.shape[1], trainXm.shape[2], trainy.shape[1]
model = Sequential()
model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax')) #^layers
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) #accuracy
model.load_weights("model_checkpoint/7/model.h5")

In [50]:
print(n_timesteps, n_features, n_outputs)

91 6 10


In [49]:
ind = 120
cnt = 0
for ind in range(425):
    new_input = dstack(np.moveaxis(trainXm[ind], -1, 0))
    real_output = trainy[ind]

    new_output = model.predict(new_input)
    ans = 0
    for i in range(10):
        if new_output[0][i] > new_output[0][ans]:
            ans = i
    if real_output[ans] != 1:
        cnt += 1
    elif ans != 2 and ans != 3:
        print(ind)
        print(ans)
        break


0
7


In [48]:
print(trainXm[0])

[[-14.49785155 -20.31543249  -3.72453493   2.88608355  -0.74083905
   -2.56278424]
 [-14.06730472 -21.94692813  -7.03637897   4.23682588  -0.50969512
   -2.21977813]
 [-13.69388178 -22.51497963 -11.82785252   5.17318527  -0.34778031
   -1.90341386]
 [-14.88548608 -21.91168256 -16.64942498   5.50979795  -0.72489749
   -1.66692935]
 [-15.19773232 -20.10333025 -17.96609317   5.23601329  -1.73690235
   -1.5284673 ]
 [-17.58885792 -19.89735868 -17.60397985   4.60749231  -2.1779386
   -1.49971594]
 [-19.20121569 -20.43717379 -16.11699862   4.01838478  -2.00537315
   -0.9319326 ]
 [-17.8174244  -20.49222861 -15.79843794   3.67323477  -1.35024851
   -0.04458444]
 [-16.45316221 -20.22833468 -16.36122682   3.25031071  -0.35103816
    0.71600895]
 [-16.69077199 -20.3691439  -16.81634181   2.643091     0.61300067
    1.3519533 ]
 [-17.05122956 -19.9833227  -17.79249581   2.16798188   0.75893952
    2.11361482]
 [-16.16238808 -18.82081484 -18.60829138   1.98474288  -0.29675126
    2.74423385]
 [-15